In [20]:
from fftarray import FFTDimension, PosArray, FreqArray, shift_frequency, shift_position
from fftarray.plot.bokeh import plt_fft
import numpy as np
from bokeh.plotting import show
from bokeh.io import output_notebook
output_notebook()

def gauss_pos(x: PosArray, a: float, sigma_pos: float) -> PosArray:
    """Normalized Gaussian distribution in position space with width sigma_pos, 
    centered around zero and scaled with a.

    Parameters
    ----------
    x : PosArray
        Incorporating the position values.
    a : float
        Scale factor. The amplitude is given by a/(sqrt(2*pi)*sigma_pos).
    sigma_pos : float
        Gaussian width.

    Returns
    -------
    PosArray
        Incorporates the function output evaluated at the input positions.
    """
    return a * np.exp(-0.5 * x**2 / sigma_pos**2) / (np.sqrt(2 * np.pi) * sigma_pos)

def gauss_freq(f: FreqArray, a: float, sigma_pos: float) -> FreqArray:
    """Gaussian distribution in frequency space with width 2*pi/sigma_pos, 
    centered around zero and amplitude a. Note that this is the Fourier 
    transform of `gauss_pos`.

    Parameters
    ----------
    f : FreqArray
        Incorporating the frequency values [ordinary units]
    a : float
        Amplitude.
    sigma_pos : float
        Gaussian width in position space.

    Returns
    -------
    FreqArray
        Incorporates the function output evaluated at the input frequencies.
    """
    return a * np.exp(-0.5 * (2*np.pi*f)**2 * sigma_pos**2)

Loading BokehJS ...

Initialize the dimension and get the basic `x` and `f` arrays. Note that the grid must fulfill the equation $N \Delta x \Delta f = 2 \pi$, where $\Delta x$ ($\Delta f$) is the step size of the position (frequency) grid. 
The `FFTDimension` makes sure that the respective `PosArray` and `FreqArray` instances incorporate values that fulfill this equation. The provided `loose_params` might be slightly adjusted during this process.

In [23]:
dim = FFTDimension("x",
    pos_middle = 1.,
    pos_extent = 10.,
    freq_middle = 2.5/(2*np.pi),
    freq_extent = 20./(2*np.pi),
    loose_params = ["pos_extent", "freq_extent"]
)

x: PosArray = dim.pos_array()
f: FreqArray = dim.freq_array()

show(plt_fft(x))
show(plt_fft(f))

In [4]:
gauss_from_pos: PosArray = gauss_pos(x = x, a = 1.2, sigma_pos = 0.7)
gauss_from_freq: FreqArray = gauss_freq(f = f, a = 1.2, sigma_pos = 0.7)

np.testing.assert_array_almost_equal(gauss_from_pos, gauss_from_freq.pos_array())
np.testing.assert_array_almost_equal(gauss_from_freq, gauss_from_pos.freq_array())

# TODO Plot titles?
show(plt_fft(gauss_from_pos))
show(plt_fft(gauss_from_freq))

Just compute a centered Gaussian in position and frequency space.
It does not matter where it is defined, the result is the same (up to numerical inaccuracies).

In [5]:
freq_shift = 0.9
gauss_from_pos: PosArray = shift_frequency(gauss_pos(x = x, a = 1.2, sigma_pos = 0.7), {"x": freq_shift})
gauss_from_freq: FreqArray = gauss_freq(f = f - freq_shift, a = 1.2, sigma_pos = 0.7)

np.testing.assert_array_almost_equal(gauss_from_pos, gauss_from_freq.pos_array())
np.testing.assert_array_almost_equal(gauss_from_freq, gauss_from_pos.freq_array())

show(plt_fft(gauss_from_pos))
show(plt_fft(gauss_from_freq))

Now test with a Gaussian shifted in position space. The `shift_position` function multiplies a phase factor unto the `FFTArray` in frequency space to achieve a user.defined shift.

In [13]:
pos_shift = 0.9
gauss_from_pos: PosArray = gauss_pos(a = 1.2, x = x - pos_shift, sigma_pos = 0.7)
gauss_from_freq: FreqArray = shift_position(gauss_freq(a = 1.2, f = f, sigma_pos = 0.7), {"x": pos_shift})

np.testing.assert_array_almost_equal(gauss_from_pos, gauss_from_freq.pos_array())
np.testing.assert_array_almost_equal(gauss_from_freq, gauss_from_pos.freq_array())

show(plt_fft(gauss_from_pos))
show(plt_fft(gauss_from_freq))